# Travelling Salesman Problem Visualization


In [1]:
import networkx as nx
import math

import random
random.seed(2333)

import numpy
numpy.random.seed(2333)

import matplotlib.pyplot as plt
from itertools import permutations

import imageio
import os

from IPython.display import Video

In [2]:
class TSP:

    def __init__(self, G):
        self.G = G
        self.pos = nx.get_node_attributes(self.G, "pos")
        self.G_ad = nx.to_numpy_array(self.G)

    def _dist_btw_nodes(n1, n2):
        return math.hypot(n1['pos'][0]-n2['pos'][0], n1['pos'][1]-n2['pos'][1])

    def visualize(self, edges=[]):
        plt.figure(figsize=(50, 25), dpi=32, frameon=True)
        plt.autoscale(enable=False)
        nx.draw_networkx(
            G=self.G,
            pos=self.pos,
            edgelist=edges,
            node_size=7200,
            font_size=40,
            width=8,
            font_color='white',
            node_color='black',
            font_weight='bold',
            edge_color='blue',
        )

    def brute_force(self):
        travelNodes = {i for i in self.G.nodes()}
        travelNodes.remove(0)  # Start at node 0

        minLength = 2**63-1
        minPath = []
        history = []

        routePermutations = permutations(travelNodes)
        for route in routePermutations:

            length = 0

            currentNode = 0
            for node in route:
                length += self.G_ad[currentNode][node]
                currentNode = node
            length += self.G_ad[currentNode][0]

            if length < minLength:
                minLength = length
                path = ([0]+list(route)+[0])
                minPath = path
                history.append(path.copy())

        return minLength, minPath, history

    def nearest_neighbour(self, startNode=None):
        travelNodes = {i for i in self.G.nodes()}
        if not startNode:
            startNode = random.choice(tuple(travelNodes))
        travelNodes.remove(startNode)  # Random startNode

        path = [startNode]
        length = 0
        currentNode = startNode
        history = [[startNode]]

        while len(travelNodes) > 0:

            next_node = None
            distance = 2**63-1

            for node in travelNodes:
                history.append(path+[node])
                if self.G_ad[currentNode][node] < distance:
                    distance = self.G_ad[currentNode][node]
                    next_node = node

            length += distance
            path.append(next_node)
            travelNodes.remove(next_node)
            currentNode = next_node

        length += self.G_ad[path[-1]][startNode]
        path.append(startNode)
        history.append(path.copy())

        return length, path, history

    def christofides(self):
        pass

    def gen_animation(self, history, name, ext='mp4'):
        if not os.path.exists(f'{name}/images'):
            os.makedirs(f'{name}/images')

        # with imageio.get_writer(f'{name}/{name}.gif', mode='I', duration=0.2) as writer:
        #     for i, v in enumerate(history):
        #         tsp.visualize(path_to_edgelist(v))
        #         plt.savefig(f'{name}/images/{i}')
        #         plt.close()
        #         writer.append_data(imageio.imread(f'{name}/images/{i}.png'))

        images = []
        for i, v in enumerate(history):
            self.visualize(path_to_edgelist(v))
            plt.savefig(f'{name}/images/{i}')
            plt.close()
            images.append(imageio.imread(f'{name}/images/{i}.png'))
        imageio.mimsave(f'{name}/{name}.{ext}', images)


In [3]:
def gen_tsp_graph(size):
    G = nx.random_geometric_graph(size, 0.2)
    pos = nx.get_node_attributes(G, "pos")
    for i in range(len(pos)):
        for j in range(i + 1, len(pos)):
            dist = math.hypot(
                pos[i][0] - pos[j][0], pos[i][1] - pos[j][1])
            G.add_edge(i, j, weight=dist)
    return G


def path_to_edgelist(path):
    return list(nx.utils.pairwise(path))


In [4]:
G = gen_tsp_graph(size=10)
tsp = TSP(G)

In [5]:
length, path, history = tsp.brute_force()
tsp.gen_animation(history, 'brute_force')
print(length, path)

3.078097894275286 [0, 3, 6, 9, 8, 5, 1, 2, 4, 7, 0]


In [6]:
Video('brute_force/brute_force.mp4')

In [7]:
length, path, history = tsp.nearest_neighbour(0)
tsp.gen_animation(history, 'nearest_neighbour')
print(length, path)

3.314333715878226 [3, 0, 7, 8, 5, 1, 2, 4, 9, 6, 3]


In [8]:
Video('nearest_neighbour/nearest_neighbour.mp4')